In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random


In [2]:
#Read our data to a Dataframe
stats_df = pd.read_csv('NBA_clean_data.csv')

In [3]:
#Get all players and amount of appearances
player_appearances = stats_df.groupby(by='Player',as_index=False).size()
#Get only those who are playing more then 1 season
players = player_appearances[player_appearances['size']>=2]
#reset index
players.reset_index(drop=True,inplace=True)
#Get the players who appear more than once in the data into a player list
players_list = players['Player'].values.tolist()

In [4]:
#Get a dataFrame containing the players from player list (move players who are only in one season)
players_df =stats_df[stats_df['Player'].isin(players_list)]
players_df.reset_index(drop=True,inplace=True)
players_df

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,2021,Steven Adams,C,27.0,NOP,58.0,58.0,1605.0,189.0,308.0,...,213.0,301.0,514.0,111.0,54.0,38.0,78.0,113.0,438.0,17.29
1,2021,Bam Adebayo,C,23.0,MIA,64.0,64.0,2143.0,456.0,800.0,...,142.0,431.0,573.0,346.0,75.0,66.0,169.0,145.0,1197.0,26.39
2,2021,LaMarcus Aldridge,C,35.0,TOT,26.0,23.0,674.0,140.0,296.0,...,19.0,99.0,118.0,49.0,11.0,29.0,27.0,47.0,352.0,18.82
3,2021,Nickeil Alexander-Walker,SG,22.0,NOP,46.0,13.0,1007.0,192.0,458.0,...,13.0,131.0,144.0,102.0,47.0,22.0,69.0,88.0,508.0,15.92
4,2021,Grayson Allen,SG,25.0,MEM,50.0,38.0,1259.0,173.0,414.0,...,19.0,141.0,160.0,108.0,46.0,8.0,48.0,71.0,532.0,15.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11709,1990,Randy Wittman,SF,30.0,IND,61.0,0.0,544.0,62.0,122.0,...,4.0,26.0,30.0,39.0,7.0,4.0,23.0,21.0,130.0,7.47
11710,1990,Joe Wolf,C,25.0,LAC,77.0,19.0,1325.0,155.0,392.0,...,63.0,169.0,232.0,62.0,30.0,24.0,77.0,129.0,370.0,7.43
11711,1990,Orlando Woolridge,SF,30.0,LAL,62.0,2.0,1421.0,306.0,550.0,...,49.0,136.0,185.0,96.0,39.0,46.0,73.0,160.0,788.0,19.81
11712,1990,James Worthy*,SF,28.0,LAL,80.0,80.0,2960.0,711.0,1298.0,...,160.0,318.0,478.0,288.0,99.0,49.0,160.0,190.0,1685.0,22.09


In [5]:
#Create a seperated Dataframe for each year
array_of_df_per_year=[]
for year in range(1990,2022):
    array_of_df_per_year.append(players_df[players_df['Year']==year])

### Adding future PER of each player up to 2020 season

In [ ]:
#Create a future PER column based on previous year(up to 2020 because we dont have the stats of 2022)
new_array_of_df_per_year =[]
for ind in range(0,len(array_of_df_per_year)-1):
    first_index = array_of_df_per_year[ind]
    second_index = array_of_df_per_year[ind+1]
    first_copy = first_index.copy()
    first_copy['Future_PER'] = np.nan
    for index,row in first_copy.iterrows():
        if row['Player'] in second_index.values:
            first_copy.at[index,'Future_PER'] = second_index[second_index['Player']== row['Player']]['PER']
    first_copy.dropna(inplace=True)
    new_array_of_df_per_year.append(first_copy)

In [ ]:
#Merge all dataframes
future_per_df = pd.concat(new_array_of_df_per_year)
#Reset Indecies
future_per_df.reset_index()
future_per_df = future_per_df.sort_values(by='Year', ascending=False)
new_array_of_df_per_year[0].head()

In [ ]:
new_array_of_df_per_year[1].head()

In [ ]:
future_per_df.info()

In [ ]:
#plt.(future_per_df['PER'],future_per_df['Future_PER'])
fig, ax = plt.subplots()
ax.scatter(future_per_df['PER'],future_per_df['Future_PER'], alpha=0.2,c = '#18A999')

ax.set_xlabel('PER', fontsize=15)
ax.set_ylabel('Future PER', fontsize=15)
ax.set_title('PER vs Future PER')

ax.grid(True)
fig.tight_layout()

plt.show()

In [ ]:
future_per_df.to_csv('NBA_future_PER_data.csv', index=False)